In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Limpieza de los datos



1. Cargar los datos

In [26]:
import pandas as pd
calificaciones = pd.read_csv("./Datos/calificaciones.csv")
estudiantes = pd.read_csv("./Datos/estudiantes.csv")
carreras = pd.read_csv("./Datos/carrera.csv")
escuelas = pd.read_csv("./Datos/escuelas.csv")

### Primero eliminamos a aquellos estudiantes que su "prepa" no esta registrada como escuela

con esto se quitaron aprox. 2000-3000 filas

In [27]:
#colonias = pd.DataFrame(estudiantes["municipio"].unique(), columns=["municipio"])
#colonias

#municipio_ciudad = estudiantes.groupby('municipio')['ciudad'].unique().reset_index()
#municipio_ciudad.columns = ['municipio', 'ciudades']
#municipio_ciudad

#if estudiantes["municipio"] > 25000 && estudiantes["municipio"] < 25019:
#    estudiantes["ciudad"]


correcta = pd.merge(estudiantes, escuelas[['clave']], left_on='prepa', right_on='clave', how='inner')
correcta.sort_values('clave_alumno')



,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida,clave
1292,1,0,1,39,Villa de Guadalupe,25006.0,CULIACAN,2003.0,2011.0,4,39
1293,2,0,1,39,Las Mañanitas,25006.0,CULIACAN,2003.0,2013.0,4,39
1294,4,0,2,36,El Walamo,25006.0,CUALIACAN,2003.0,2061.0,4,36
1295,5,0,2,58,Las Granjas,25006.0,CULIACAN,2003.0,2011.0,4,58
1296,6,0,1,31,FIDEL VELAZQUEZ,25006.0,"ELDORADO, CLN",2003.0,2051.0,5,31
...,...,...,...,...,...,...,...,...,...,...,...
4409,11956,0,1,59,Gustavo Diaz Ordaz,25002.0,ANGOSTURA,2111.0,NaN,1,59
4410,11957,0,1,59,Gustavo Diaz Ordaz,25002.0,ANGOSTURA,2111.0,NaN,1,59
4411,11960,0,1,89,Humaya,25006.0,CULIACAN,2111.0,NaN,1,89
4412,11962,0,1,22,4 de Marzo,25006.0,CULIACAN,2111.0,NaN,1,22


### Ahora se filtran todas las calificaciones de alumnos, si la clave del alumno no esta registrada en alumnos se descartan.

In [28]:
correcta2 = pd.merge(calificaciones, estudiantes[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
correcta2.sort_values('clave_alumno')

,periodo,clave_alumno,clave_materia,calificacion,tipo_calificacion
37645,2011,1,04tBt3,80.0,1
37639,2003,1,04tGt4,100.0,1
37640,2003,1,04tBt2,85.0,1
37641,2003,1,04tBt1,85.0,1
37642,2003,1,04tGt5,0.0,3
...,...,...,...,...,...
374650,2103,11853,INN1008,100.0,1
374651,2103,11853,ACA0907,80.0,1
374652,2103,11853,ACC0906,95.0,1
374653,2103,11853,ACF0901,95.0,1


### aqui filtro a aquellos estudiantes que necesitamos solamente, aquellos que siguen vigentes y que no han terminado sus estudios

In [30]:
correcta3 = estudiantes[(estudiantes['termino'].isnull()) & (estudiantes['causa_salida']==1)]
correcta3.sort_values('clave_alumno')
correcta3

,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida
607,3327,4,1,121,BUENOS,25006.0,CULIACAN,2043.0,NaN,1
648,4420,4,1,154,El Walamo,25011.0,GUASAVE,2053.0,NaN,1
662,4434,4,1,127,Alcanfores,25018.0,NAVOLATO,2053.0,NaN,1
668,4440,4,1,117,Los Angeles,25006.0,CULIACAN,2053.0,NaN,1
671,4443,4,1,142,NaN,25002.0,LA REFORMA ANG.,2053.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
11930,11738,8,2,98,Fovissste Playa Azul,25006.0,CULIACAN,2103.0,NaN,1
11931,11739,8,1,91,LA,25006.0,CULIACAN,2103.0,NaN,1
11932,11740,8,1,71,BALCONES,25006.0,CULIACAN,2103.0,NaN,1
11933,11825,8,1,26,San Miguel,25006.0,CULIACAN,2103.0,NaN,1


In [31]:
#los municipios con clave le asignamos su ciudad correspondiente
correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if x == 25019 else 'Juan Jose Rios' if x == 25020 else 'No encontrado')
#para los municipios que no tienen bien su clave se le asignan por default a culiacan
clave_municipios = [25001, 25002, 25003, 25004, 25005, 25006, 25007, 25008, 25009, 25010, 25011, 25012, 25013, 25014, 25015, 25016, 25017, 25018, 25019, 25020]
clave_cul = 25006
#si no esta en la lista de claves de municipios se le asigna a culiacan
correcta3.loc[~correcta3['municipio'].isin(clave_municipios), 'ciudad'] = 'Culiacan'
#ahora cambiamos los municipios que estan mal a la clave de culiacan
correcta3['municipio'] = correcta3['municipio'].apply(lambda x: clave_cul if x not in clave_municipios else x)

#poner todo en mayusculas
correcta3['ciudad'] = correcta3['ciudad'].str.upper()
correcta3['colonia'] = correcta3['colonia'].str.upper()

/var/folders/rf/nsp4kq195572kp5xlt38jh380000gn/T/ipykernel_12482/2394312866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if 

In [33]:
correcta3.to_csv("./Datos/correcta3ss.csv", index=False)

In [34]:
#correcta4 = pd.merge(calificaciones, correcta3[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
#correcta4.sort_values('clave_alumno')

correcta3['colonia'] = correcta3['colonia'].replace({
    '21': '21 DE MARZO', 
    '21DE': '21 DE MARZO',
    'ADF': 'ADOLFO LOPEZ MATEOS',
    'ADOLFO': 'ADOLFO LOPEZ MATEOS',
    'AF': 'ADOLFO LOPEZ MATEOS',
    'AGRI.': 'COLONIA AGRICOLA MEXICO',
    'AGRICOLA': 'COLONIA AGRICOLA MEXICO',
    'COL.AGRICOLA': 'COLONIA AGRICOLA MEXICO',
    'AGUA': 'AGUACALIENTE',
    'AGUA CALIENTE': 'AGUACALIENTE',
    'ALFONSO': 'ALFONSO G. CALDERON',
    'AMP.TOLEDO': 'AMP. TOLEDO',
    'ALTURAS': 'ALTURAS DEL SUR',
    'AMADO': 'AMADO NERVO',
    'AQUILES': 'AQUILES SERDAN',
    'BALCONES': 'BALCONES DE NUEVO CULIACAN',
    'BUENOS': 'BUENOS AIRES',
    'BUENA': 'BUENA VISTA',
    'CNOP': 'AMP. CNOP',
    'AMPL.CNOP': 'AMP. CNOP',
    'CU': 'CD. UNIVERSITARIA',
    'C.U': 'CD. UNIVERSITARIA',
    'CIUDADES': 'CIUDADES HERMANAS',
    'CD.': 'CIUDADES HERMANAS',
    'CD. HERMANAS': 'CIUDADES HERMANAS',
    'CHULA': 'CHULAVISTA',
    'CHULA VISTA': 'CHULAVISTA',
    'CONOCIDO': 'CONOCIDA',
    'CONOSIDO': 'CONOCIDA',
    'CONOSIDA': 'CONOCIDA',
    'FRAC.NAKAYAMA': 'FRACC. NAKAYAMA',
    'FRAC.': 'FRACC.',
    'FRA': 'FRACC.',
    'FRACC.-': 'FRACC.',
    'EJ.': 'EJIDAL',
    'E.': 'EJIDAL',
    'CANACO': 'FRACC. CANACO',
    'FRACC.CANACO': 'FRACC. CANACO',
    'ESCOBEDO': 'COL. MARIANO ESCOBEDO',
    'ESCOBESO': 'COL. MARIANO ESCOBEDO',
    'FDEL GUTIERREZ': 'FIDEL GUTIERREZ',
    'ZAPATA': 'EMILIANO ZAPATA',
    'GENARO': 'COL. GENARO ESTRADA',
    'GUADALUPE': 'GUADALUPE VICTORIA',
    'GPE.': 'GUADALUPE VICTORIA',
    'HIGUERAS DE CAMPA√ëA': 'HIGUERAS DE CAMPAÑA',
    'INF.': 'INFONAVIT',
    'INF': 'INFONAVIT',
    'IND.': 'INDUSTRIAL EL PALMITO',
    'IND.EL': 'INDUSTRIAL EL PALMITO',
    'INF.LAS': 'INFONAVIT LAS FLORES',
    'BARRANCOS': 'INFONAVIT BARRANCOS',
    'INF.BARRANACOS': 'INFONAVIT BARRANCOS',
    'JORGE': 'JORGE ALMADA',
    'JUAN': 'JUAN DE DIOS BATIZ',
    'JUANDEDIOSBATIZ': 'JUAN DE DIOS BATIZ',
    'L√ÅZARO': 'LAZARO CARDENAS',
    'LAS MA√ëANITAS': 'LAS MAÑANITAS',
    'NUEVO': 'NUEVO CULIACAN',
    'LOMALINDA': 'LOMA LINDA',
    'LINDA': 'LINDA VISTA',
    'PLUTARCO': 'PLUTARCO ELIAS CALLES',
    'NI√ëOS HEROES': 'NIÑOS HEROES',
    'REC.': 'RECURSOS',
    'REPUBLICA': 'REPUBLICA MEXICANA',
    'REP.': 'REPUBLICA MEXICANA',
    'REPUBLIA': 'REPUBLICA MEXICANA',
    'URBI': 'URBI QUINTA VERSALLES',
    'URBIQUINTA': 'URBI QUINTA VERSALLES',
    'VICENTE': 'VICENTE LOMBARDO TOLEDANO',
    'SIND.': 'SINDICATURA TEPUCHE',
    'SIND. TEPUCHE': 'SINDICATURA TEPUCHE',
    'STANZA': 'STANZA SOLARE',
    'MONTE': 'FRACC. MONTEBELLO',
    'MONTE BELLO': 'FRACC. MONTEBELLO'
})


/var/folders/rf/nsp4kq195572kp5xlt38jh380000gn/T/ipykernel_12482/1411759601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['colonia'] = correcta3['colonia'].replace({
